In [113]:
import pandas as pd

In [114]:
df_bpi = pd.read_json(r'sources/bpifrance_startups_data2.json')
df_tech = pd.read_json(r"sources/tech_fest_data.json")
df_maddy = pd.read_json(r"sources/entreprises_data - maddyness.json")
df_CESFR = pd.read_csv(r"sources/exposantsFRCES2025.csv")
df_mina = pd.read_csv(r'sources/societes_minalogic.csv')

In [115]:
# Débalage des colonnes

import ast

def to_dict(x):
    if isinstance(x, str):
        try:
            return ast.literal_eval(x)
        except:
            return None
    return x

df_mina['contact'] = df_mina['contact'].apply(to_dict)
df_mina['infos'] = df_mina['infos'].apply(to_dict)

df_mina['Adresse'] = df_mina['contact'].apply(lambda d: d.get('Adresse') if isinstance(d, dict) else None)
df_mina['Contact'] = df_mina['contact'].apply(lambda d: d.get('Contact') if isinstance(d, dict) else None)
df_mina['Date de création'] = df_mina['infos'].apply(lambda d: d.get('Date de création') if isinstance(d, dict) else None)
df_mina['mots_cles_a'] = df_mina['infos'].apply(lambda d: d.get('Thématiques') if isinstance(d, dict) else None)
df_mina['Marché'] = df_mina['infos'].apply(lambda d: d.get('Marchés') if isinstance(d, dict) else None)
df_mina["Date d'adhésion"] = df_mina['infos'].apply(lambda d: d.get("Date d'adhésion") if isinstance(d, dict) else None)
df_mina["Type d'organisme"] = df_mina['infos'].apply(lambda d: d.get("Type d'organisme") if isinstance(d, dict) else None)

In [116]:
#Renomage de la colonne name en nom
df_bpi = df_bpi.rename(columns={'name': 'nom'})
df_bpi = df_bpi.rename(columns={'hashtags': 'mots_cles_b'})
df_bpi = df_bpi.rename(columns={'website': 'site_web'})

df_CESFR = df_CESFR.rename(columns={'Nom': 'nom'})
df_CESFR = df_CESFR.rename(columns={'Description': 'description'})
df_CESFR = df_CESFR.rename(columns={'Catégories': 'mots_cles_y'})
df_CESFR = df_CESFR.rename(columns={'Website': 'site_web_z'})
df_CESFR = df_CESFR.rename(columns={'Adresse': 'adresse_z'})
df_CESFR = df_CESFR.rename(columns={'Logo': 'logo'})


df_maddy = df_maddy.rename(columns={'Nom': 'nom'})
df_maddy = df_maddy.rename(columns={'Description': 'description'})
df_maddy = df_maddy.rename(columns={'Site internet': 'site_web'})
df_maddy = df_maddy.rename(columns={'Logo': 'logo'})
df_maddy = df_maddy.rename(columns={'Hashtags': 'mots_cles_z'})

df_mina = df_mina.rename(columns={'name': 'nom'})
df_mina = df_mina.rename(columns={'Logo': 'logo'})


In [117]:
#Suppression colonne
df_tech.drop(columns=['social_links'], inplace=True)
df_bpi.drop(columns=['total_funding'], inplace=True)

df_CESFR.drop(columns=['Lien'], inplace=True)

df_maddy.drop(columns=['Siège'], inplace=True)
df_maddy.drop(columns=['Date de création'], inplace=True)

df_mina.drop(columns=['url'], inplace=True)
df_mina.drop(columns=["Date d'adhésion"], inplace=True)
df_mina.drop(columns=["contact"], inplace=True)
df_mina.drop(columns=["infos"], inplace=True)

In [118]:
# Convertir les colonnes 'nom' en majuscules
df_bpi['nom'] = df_bpi['nom'].str.upper()
df_tech['nom'] = df_tech['nom'].str.upper()
df_maddy['nom'] = df_maddy['nom'].str.upper()
df_CESFR['nom'] = df_CESFR['nom'].str.upper()
df_mina['nom'] = df_mina['nom'].str.upper()

In [119]:
# Fusion des 3 DataFrames sur la colonne 'nom'

# Fusion df_bpi et df_tech
merged_df = pd.merge(df_bpi, df_tech, on=['nom','description','logo'], how='outer')

# Fusion de la merge avec df_maddy
merged_df = pd.merge(merged_df, df_maddy, on=['nom','description','logo'], how='outer')

# Fusion de la merge avec df_maddy
merged_df = pd.merge(merged_df, df_CESFR, on=['nom','description','logo'], how='outer')

# Fusion de la merge avec df_mina
merged_df = pd.merge(merged_df, df_mina, on=['nom','description','logo'], how='outer')

In [120]:
merged_df.columns

Index(['nom', 'description', 'logo', 'product_types', 'sectors', 'city',
       'site_web_x', 'mots_cles_b', 'mots_cles', 'site_web_y', 'tags',
       'emplacement', 'site_web', 'Date dernier financement', 'Série',
       'Montant', 'mots_cles_z', 'mots_cles_y', 'site_web_z', 'LinkedIn',
       'adresse_z', 'Adresse', 'Contact', 'Date de création', 'mots_cles_a',
       'Marché', 'Type d'organisme'],
      dtype='object')

In [121]:
#remplissage des bonnes colonnes

merged_df['site_web_x'].fillna(merged_df['site_web_y'], inplace=True)
merged_df['site_web_x'].fillna(merged_df['site_web_z'], inplace=True)
merged_df['site_web_x'].fillna(merged_df['site_web'], inplace=True)

merged_df['mots_cles_b'].fillna(merged_df['mots_cles'], inplace=True)
merged_df['mots_cles_b'].fillna(merged_df['mots_cles_z'], inplace=True)
merged_df['mots_cles_b'].fillna(merged_df['mots_cles_y'], inplace=True)
merged_df['mots_cles_b'].fillna(merged_df['mots_cles_a'], inplace=True)

merged_df['adresse_z'].fillna(merged_df['city'], inplace=True)
merged_df['adresse_z'].fillna(merged_df['Adresse'], inplace=True)

In [122]:
merged_df.columns

Index(['nom', 'description', 'logo', 'product_types', 'sectors', 'city',
       'site_web_x', 'mots_cles_b', 'mots_cles', 'site_web_y', 'tags',
       'emplacement', 'site_web', 'Date dernier financement', 'Série',
       'Montant', 'mots_cles_z', 'mots_cles_y', 'site_web_z', 'LinkedIn',
       'adresse_z', 'Adresse', 'Contact', 'Date de création', 'mots_cles_a',
       'Marché', 'Type d'organisme'],
      dtype='object')

In [123]:
#Suppression de la colonne description_y
merged_df.drop(columns=['site_web_y'], inplace=True)
merged_df.drop(columns=['site_web_z'], inplace=True)
merged_df.drop(columns=['site_web'], inplace=True)

merged_df.drop(columns=['mots_cles'], inplace=True)
merged_df.drop(columns=['mots_cles_z'], inplace=True)
merged_df.drop(columns=['mots_cles_y'], inplace=True)
merged_df.drop(columns=['mots_cles_a'], inplace=True)

merged_df.drop(columns=['city'], inplace=True)
merged_df.drop(columns=['Adresse'], inplace=True)


In [127]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7477 entries, 0 to 7476
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   nom                       7475 non-null   object
 1   description               7473 non-null   object
 2   logo                      7475 non-null   object
 3   product_types             4364 non-null   object
 4   sectors                   4364 non-null   object
 5   site_web_x                5766 non-null   object
 6   mots_cles_b               7451 non-null   object
 7   tags                      260 non-null    object
 8   emplacement               260 non-null    object
 9   Date dernier financement  1697 non-null   object
 10  Série                     1697 non-null   object
 11  Montant                   1697 non-null   object
 12  LinkedIn                  166 non-null    object
 13  adresse_z                 4855 non-null   object
 14  Contact                 

In [126]:
merged_df

,nom,description,logo,product_types,sectors,site_web_x,mots_cles_b,tags,emplacement,Date dernier financement,Série,Montant,LinkedIn,adresse_z,Contact,Date de création,Marché,Type d'organisme
0,INVENTY,Inventy est une scale up de technologies innov...,https://upload.prd.hbd.cloud.bpifrance.fr/4826...,"[B2B, Saas, , , , , , , , , , , ]","[B2B, Saas, , , , , , , , , , , ]",http://www.inventy.com/,[],NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN
1,360LEARNING,360Learning permet aux équipes Formation de pi...,https://upload.prd.hbd.cloud.bpifrance.fr/awsS...,"[, , , , , , , , , , , ]","[, , , , , , , , , , , ]",https://fr.360learning.com/,"[#Education, #EdTech, #MOOC, #Communities, #Co...",NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN
2,KLAXOON,Une suite unique de 10 outils de collaboration...,https://upload.prd.hbd.cloud.bpifrance.fr/08f8...,"[, , , , , , , , , , , , , , , , , , , , , , ,...","[, , , , , , , , , , , , , , , , , , , , , , ,...",https://klaxoon.com/fr/,"[#EdTech, #E-Learning, #gestion de projet, #ré...",NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN
3,WELCOME TO THE JUNGLE,Welcome to the Jungle est le média de référenc...,https://upload.prd.hbd.cloud.bpifrance.fr/star...,"[, , , , , , , , , , , , , ]","[, , , , , , , , , , , , , ]",https://www.welcometothejungle.com/fr,"[#Staffing Agency, #SaaS, #Human Resources]",NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN
4,SIGHTCALL,SightCall est le leader mondial de l’Augmented...,https://upload.prd.hbd.cloud.bpifrance.fr/star...,"[, , , , , , , , , , , , , , , , ]","[, , , , , , , , , , , , , , , , ]",https://sightcall.com/fr/,[],NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7472,SAT4SPACE,SAT4SPACE est une entreprise qui propose des s...,https://www.minalogic.com/wp-content/uploads/2...,NaN,NaN,NaN,"Logiciel, ...",NaN,NaN,NaN,NaN,NaN,NaN,"Bât. Startway, 1 place F. Roosevelt\n ...",NOWICKI-BRINGUIER Yoanna-Reine ...,2021,"Chimie Environnement, ...",PME
7473,LA NOTE BLEUE,LA MAISON\nArts Vivants - Expériences Immersiv...,https://www.minalogic.com/wp-content/uploads/2...,NaN,NaN,NaN,Non Applicable,NaN,NaN,NaN,NaN,NaN,NaN,20 Rue de la Grenière\n ...,DOUROUX Philippe ...,2007,ICC,PME
7474,DELTA CONCEPT,"La société, son activité :\nPour tous ceux qui...",https://www.minalogic.com/wp-content/uploads/2...,NaN,NaN,NaN,Non Applicable,NaN,NaN,NaN,NaN,NaN,NaN,1 rue de l'Isère\n ...,PITONE Gilbert ...,2006,"Energie Bâtiment, ...",PME
7475,CIC LYONNAISE DE BANQUE,"CIC Lyonnaise de Banque, Banque du groupe CIC,...",https://www.minalogic.com/wp-content/uploads/2...,NaN,NaN,NaN,Non Applicable,NaN,NaN,NaN,NaN,NaN,NaN,8 rue du bat d'argent\n ...,BOULANGER Maude ...,1931,Non applicable,Banque & Investisseur


In [78]:
import pandas as pd

def combine_strings(series):
    """Retourne la première chaîne non vide/nulle."""
    for val in series:
        if isinstance(val, str) and val.strip() != "":
            return val
    return None

def combine_lists(series):
    """Fusionne toutes les listes en un set (pour éviter les doublons), puis retransforme en liste."""
    combined = set()
    for val in series.dropna():
        if isinstance(val, list):
            combined.update([x for x in val if x])  # on évite les chaînes vides
    return list(combined) if combined else None

def combine_first_nonnull(series):
    """Renvoie la première valeur non-nulle, quel que soit son type."""
    for val in series:
        if pd.notnull(val):
            return val
    return None


aggregation_dict = {
    'nom': 'first',  # on peut prendre n’importe laquelle, elles sont toutes identiques dans le groupe
    'description': combine_strings,
    'logo': combine_strings,
    'mots_cles_b': combine_lists,
    'site_web_x': combine_strings,
    'tags': combine_lists,
    'emplacement': combine_strings,  # ou combine_first_nonnull, etc.
    'Date dernier financement': combine_strings,
    'Série': combine_strings,
    'Montant': combine_strings,
    'LinkedIn': combine_strings,
    'adresse_z': combine_strings,
    'Contact': combine_strings,
    'Date de création': combine_strings,
    'Marché': combine_strings,
    "Type d'organisme": combine_strings

    # Ajoutez toutes les colonnes nécessaires
}

In [79]:
merged_df

,nom,description,logo,product_types,sectors,site_web_x,mots_cles_b,tags,emplacement,Date dernier financement,Série,Montant,LinkedIn,adresse_z,Contact,Date de création,Marché,Type d'organisme
0,INVENTY,Inventy est une scale up de technologies innov...,https://upload.prd.hbd.cloud.bpifrance.fr/4826...,"[B2B, Saas, , , , , , , , , , , ]","[B2B, Saas, , , , , , , , , , , ]",http://www.inventy.com/,[],NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN
1,360LEARNING,360Learning permet aux équipes Formation de pi...,https://upload.prd.hbd.cloud.bpifrance.fr/awsS...,"[, , , , , , , , , , , ]","[, , , , , , , , , , , ]",https://fr.360learning.com/,"[#Education, #EdTech, #MOOC, #Communities, #Co...",NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN
2,KLAXOON,Une suite unique de 10 outils de collaboration...,https://upload.prd.hbd.cloud.bpifrance.fr/08f8...,"[, , , , , , , , , , , , , , , , , , , , , , ,...","[, , , , , , , , , , , , , , , , , , , , , , ,...",https://klaxoon.com/fr/,"[#EdTech, #E-Learning, #gestion de projet, #ré...",NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN
3,WELCOME TO THE JUNGLE,Welcome to the Jungle est le média de référenc...,https://upload.prd.hbd.cloud.bpifrance.fr/star...,"[, , , , , , , , , , , , , ]","[, , , , , , , , , , , , , ]",https://www.welcometothejungle.com/fr,"[#Staffing Agency, #SaaS, #Human Resources]",NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN
4,SIGHTCALL,SightCall est le leader mondial de l’Augmented...,https://upload.prd.hbd.cloud.bpifrance.fr/star...,"[, , , , , , , , , , , , , , , , ]","[, , , , , , , , , , , , , , , , ]",https://sightcall.com/fr/,[],NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7472,SAT4SPACE,SAT4SPACE est une entreprise qui propose des s...,https://www.minalogic.com/wp-content/uploads/2...,NaN,NaN,NaN,"Logiciel, ...",NaN,NaN,NaN,NaN,NaN,NaN,"Bât. Startway, 1 place F. Roosevelt\n ...",NOWICKI-BRINGUIER Yoanna-Reine ...,2021,"Chimie Environnement, ...",PME
7473,LA NOTE BLEUE,LA MAISON\nArts Vivants - Expériences Immersiv...,https://www.minalogic.com/wp-content/uploads/2...,NaN,NaN,NaN,Non Applicable,NaN,NaN,NaN,NaN,NaN,NaN,20 Rue de la Grenière\n ...,DOUROUX Philippe ...,2007,ICC,PME
7474,DELTA CONCEPT,"La société, son activité :\nPour tous ceux qui...",https://www.minalogic.com/wp-content/uploads/2...,NaN,NaN,NaN,Non Applicable,NaN,NaN,NaN,NaN,NaN,NaN,1 rue de l'Isère\n ...,PITONE Gilbert ...,2006,"Energie Bâtiment, ...",PME
7475,CIC LYONNAISE DE BANQUE,"CIC Lyonnaise de Banque, Banque du groupe CIC,...",https://www.minalogic.com/wp-content/uploads/2...,NaN,NaN,NaN,Non Applicable,NaN,NaN,NaN,NaN,NaN,NaN,8 rue du bat d'argent\n ...,BOULANGER Maude ...,1931,Non applicable,Banque & Investisseur


In [80]:
merged_df = merged_df.groupby('nom', as_index=False).agg(aggregation_dict)

In [81]:
#Identification des doublons 
merged_df

,nom,description,logo,mots_cles_b,site_web_x,tags,emplacement,Date dernier financement,Série,Montant,LinkedIn,adresse_z,Contact,Date de création,Marché,Type d'organisme
0,#DOT:DOT,#dot:dot est un réseau mobile WiFi maillé oppo...,https://www.minalogic.com/wp-content/uploads/2...,None,None,None,None,None,None,None,None,9 Avenue Paul Verlaine\n ...,BUREAU Christophe ...,2021,"Infrastructures du numérique, ...",PME
1,1001PHARMACIES,Achetez sur internet chez les pharmacies franç...,https://upload.prd.hbd.cloud.bpifrance.fr/star...,None,https://www.1001pharmacies.com/,None,None,None,None,None,None,None,None,None,None,None
2,100M,100M simplifie la production du reporting trad...,https://upload.prd.hbd.cloud.bpifrance.fr/star...,None,None,None,None,None,None,None,None,None,None,None,None,None
3,10H11,10h11 est une société technologique dans l’opt...,https://upload.prd.hbd.cloud.bpifrance.fr/star...,None,https://www.10h11.com/,None,None,None,None,None,None,Bordeaux,None,None,None,None
4,14H28,Agence de marketing numérique,https://upload.prd.hbd.cloud.bpifrance.fr/d09a...,None,https://14h28.com/,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6674,ÉDIFICE,Open Digital Education - Reinventing school wi...,https://upload.prd.hbd.cloud.bpifrance.fr/star...,None,None,None,None,None,None,None,None,None,None,None,None,None
6675,ÉLECOCITÉ,Fournisseur d'électricité d' IMPACT et platefo...,https://upload.prd.hbd.cloud.bpifrance.fr/a387...,"[#economieenergie, #crowdfunding, #transitione...",https://www.elecocite.fr/,None,None,None,None,None,None,None,None,None,None,None
6676,ÊVERIE,Êverie = Concepteur de produits de rupture pou...,https://upload.prd.hbd.cloud.bpifrance.fr/star...,None,http://www.everie.com/,None,None,None,None,None,None,None,None,None,None,None
6677,ŌBERON,"Issue de la recherche publique (CEA/CNRS), Ōbe...",https://cdn-assets.inwink.com/e3d94d99-ffd5-4d...,"[RSE, tech&planet, tech&solutions, TERRITOIRES]",Site web non disponible,None,Emplacement non spécifié,None,None,None,None,None,None,None,None,None


In [111]:
merged_df.describe().T

,count,unique,top,freq
nom,6679,6679,#DOT:DOT,1
description,6664,6610,Description non disponible,31
logo,6679,6592,https://cdn-assets.inwink.com/backoffice-publi...,44
mots_cles_b,1325,758,[#medtech],41
site_web_x,5305,5002,Site web non disponible,248
tags,43,14,[5- Global & stratégie],15
emplacement,260,13,Emplacement non spécifié,126
Date dernier financement,1694,137,10.01.21,53
Série,1515,19,Amorçage,884
Montant,1694,298,1.0 M€,124


In [82]:
#Supression des doublons 
merged_df = merged_df.drop_duplicates(subset=['nom'], keep='first')

In [83]:
merged_df

,nom,description,logo,mots_cles_b,site_web_x,tags,emplacement,Date dernier financement,Série,Montant,LinkedIn,adresse_z,Contact,Date de création,Marché,Type d'organisme
0,#DOT:DOT,#dot:dot est un réseau mobile WiFi maillé oppo...,https://www.minalogic.com/wp-content/uploads/2...,None,None,None,None,None,None,None,None,9 Avenue Paul Verlaine\n ...,BUREAU Christophe ...,2021,"Infrastructures du numérique, ...",PME
1,1001PHARMACIES,Achetez sur internet chez les pharmacies franç...,https://upload.prd.hbd.cloud.bpifrance.fr/star...,None,https://www.1001pharmacies.com/,None,None,None,None,None,None,None,None,None,None,None
2,100M,100M simplifie la production du reporting trad...,https://upload.prd.hbd.cloud.bpifrance.fr/star...,None,None,None,None,None,None,None,None,None,None,None,None,None
3,10H11,10h11 est une société technologique dans l’opt...,https://upload.prd.hbd.cloud.bpifrance.fr/star...,None,https://www.10h11.com/,None,None,None,None,None,None,Bordeaux,None,None,None,None
4,14H28,Agence de marketing numérique,https://upload.prd.hbd.cloud.bpifrance.fr/d09a...,None,https://14h28.com/,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6674,ÉDIFICE,Open Digital Education - Reinventing school wi...,https://upload.prd.hbd.cloud.bpifrance.fr/star...,None,None,None,None,None,None,None,None,None,None,None,None,None
6675,ÉLECOCITÉ,Fournisseur d'électricité d' IMPACT et platefo...,https://upload.prd.hbd.cloud.bpifrance.fr/a387...,"[#economieenergie, #crowdfunding, #transitione...",https://www.elecocite.fr/,None,None,None,None,None,None,None,None,None,None,None
6676,ÊVERIE,Êverie = Concepteur de produits de rupture pou...,https://upload.prd.hbd.cloud.bpifrance.fr/star...,None,http://www.everie.com/,None,None,None,None,None,None,None,None,None,None,None
6677,ŌBERON,"Issue de la recherche publique (CEA/CNRS), Ōbe...",https://cdn-assets.inwink.com/e3d94d99-ffd5-4d...,"[RSE, tech&planet, tech&solutions, TERRITOIRES]",Site web non disponible,None,Emplacement non spécifié,None,None,None,None,None,None,None,None,None


In [84]:
# Exporter en CSV avec un point-virgule comme séparateur et un encodage UTF-8
merged_df.to_csv("merged_df.csv", sep=';', encoding='utf-8-sig', index=False)

In [107]:
import pandas as pd
import csv
import uuid

# Charger le fichier CSV source
file_path = "merged_df.csv"  # Remplace avec le chemin de ton fichier

# Forcer le séparateur à point-virgule (éviter détection automatique)
detected_delimiter = ";"

# Charger le CSV avec des options robustes
df = pd.read_csv(file_path, delimiter=detected_delimiter, engine="python", quoting=csv.QUOTE_NONE, on_bad_lines='skip', encoding="utf-8-sig")

# Afficher les 5 premières lignes après lecture pour vérifier la structure
print("Aperçu des données après lecture :\n", df.head())

# Afficher les noms des colonnes après lecture
print("Colonnes détectées après correction :", df.columns.tolist())

# Normaliser les noms de colonnes (supprimer espaces et BOM éventuel)
df.columns = df.columns.str.strip()

# ---- Création de la table `companies` ---- #
expected_columns = ["nom", "description", "logo", "site_web_x", "Date de création", "Marché", "mots_cles_b", "adresse_z", "emplacement"]
available_columns = [col for col in expected_columns if col in df.columns]
companies_df = df[available_columns].copy()

# Renommer les colonnes pour correspondre au schéma SQL
rename_dict = {
    "nom": "name", "description": "description", "logo": "logo_url", "site_web_x": "website",
    "Date de création": "founded_date", "Marché": "category", "mots_cles_b": "subcategories",
    "adresse_z": "address", "emplacement": "city"
}
companies_df.rename(columns=rename_dict, inplace=True)

# Correction des dates incomplètes et valeurs vides
if "founded_date" in companies_df.columns:
    companies_df["founded_date"] = companies_df["founded_date"].replace(["", "None", "NaN"], pd.NaT)
    companies_df["founded_date"] = companies_df["founded_date"].apply(lambda x: x + "-01-01" if isinstance(x, str) and len(x) == 4 else x)
    companies_df["founded_date"] = pd.to_datetime(companies_df["founded_date"], errors='coerce')

# Supprimer les doublons en gardant la première occurrence
companies_df = companies_df.drop_duplicates(subset=["name"], keep="first")

# Générer un UUID pour chaque entreprise
companies_df.insert(0, "id", [str(uuid.uuid4()) for _ in range(len(companies_df))])

# Sauvegarde
companies_path = "companies.csv"
companies_df.to_csv(companies_path, index=False)

# Vérifier si 'name' est bien dans companies_df avant de créer le dictionnaire
if "name" in companies_df.columns:
    company_uuid_map = dict(zip(companies_df["name"], companies_df["id"]))
    # Supprimer les entrées avec NaN
    company_uuid_map = {k: v for k, v in company_uuid_map.items() if pd.notna(v)}
else:
    print("Erreur : La colonne 'name' est absente après suppression des doublons !")
    company_uuid_map = {}

# ---- Création de la table `people` ---- #
expected_columns_people = ["nom", "Contact", "LinkedIn"]
available_columns_people = [col for col in expected_columns_people if col in df.columns]
people_df = df[available_columns_people].copy()

# Renommer les colonnes
people_df.rename(columns={"nom": "company_id", "Contact": "full_name", "LinkedIn": "linkedin_url"}, inplace=True)

# Remplacer les noms d'entreprises par leur UUID
people_df["company_id"] = people_df["company_id"].map(company_uuid_map)

# Vérifier les entreprises manquantes dans companies.csv
missing_companies = people_df[~people_df["company_id"].isin(companies_df["id"])]
print("❌ Entreprises manquantes dans companies.csv :", len(missing_companies))
if not missing_companies.empty:
    print(missing_companies[["company_id"]].drop_duplicates())

# Supprimer les références à des entreprises inexistantes
people_df = people_df[people_df["company_id"].isin(companies_df["id"])]
people_df = people_df.dropna(subset=["company_id"])

# Séparer `full_name` en `first_name` et `last_name`
if "full_name" in people_df.columns:
    people_df[['first_name', 'last_name']] = people_df['full_name'].str.split(' ', 1, expand=True)
    people_df.drop(columns=['full_name'], inplace=True)

# Vérifier le nombre de lignes finales
print("✅ Nombre final de lignes dans people.csv :", len(people_df))

# Sauvegarde
people_path = "people.csv"
people_df.to_csv(people_path, index=False)

# ---- Création de la table `funding_rounds` ---- #
expected_columns_funding = ["nom", "Date dernier financement", "Série", "Montant"]
available_columns_funding = [col for col in expected_columns_funding if col in df.columns]
funding_rounds_df = df[available_columns_funding].copy()

# Renommer les colonnes
funding_rounds_df.rename(columns={"nom": "company_id", "Date dernier financement": "date", "Série": "round_type", "Montant": "amount"}, inplace=True)

# Correction des dates incomplètes et valeurs vides
if "date" in funding_rounds_df.columns:
    funding_rounds_df["date"] = funding_rounds_df["date"].replace(["", "None", "NaN", "NULL"], pd.NaT)
    funding_rounds_df["date"] = funding_rounds_df["date"].apply(lambda x: x + "-01-01" if isinstance(x, str) and len(x) == 4 else x)
    funding_rounds_df["date"] = pd.to_datetime(funding_rounds_df["date"], errors='coerce')

# Vérifier si des valeurs vides existent avant l'exportation
print("Valeurs vides dans 'date':", funding_rounds_df["date"].isna().sum())

# Remplacer les noms d'entreprises par leur UUID
funding_rounds_df["company_id"] = funding_rounds_df["company_id"].map(company_uuid_map)

# Sauvegarde
funding_rounds_path = "funding_rounds.csv"
funding_rounds_df.to_csv(funding_rounds_path, index=False)

print("Les fichiers ont été créés avec succès :")
print(f"- {companies_path}")
print(f"- {people_path}")
print(f"- {funding_rounds_path}")


Aperçu des données après lecture :
                                                  nom  \
0                                           #DOT:DOT   
1                                                ...   
2                                                ...   
3                                     1001PHARMACIES   
4                                               100M   

                                         description  \
0  "#dot:dot est un réseau mobile WiFi maillé opp...   
1                                               None   
2  BUREAU Christophe                             ...   
3  Achetez sur internet chez les pharmacies franç...   
4  100M simplifie la production du reporting trad...   

                                                logo  \
0   (ii) le répéteur WiFi auto-reconfigurable, ét...   
1                                               None   
2                                               2021   
3  https://upload.prd.hbd.cloud.bpifrance.fr/star...   
4  https:/

C:\Users\cohen\AppData\Local\Temp\ipykernel_24120\2199307565.py:84: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  people_df[['first_name', 'last_name']] = people_df['full_name'].str.split(' ', 1, expand=True)
